This notebook create dataset config files from reference

In [40]:
import pandas as pd
import os
import glob
import json

In [41]:
os.getcwd()

'/neurospin/dico/jchavas/Runs/70_self-supervised_two-regions/Program/2023_jlaval_STSbabies/contrastive/notebooks/joel/utils'

In [42]:
path = f"{os.getcwd()}/../../../configs/dataset/with_reskel_distbottom/2mm/UKB"
ref_file = f"{path}/reference.yaml"
crop_path = "/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/crops/2mm"
crop_dirs = glob.glob(f"{crop_path}/*")
crop_drops = ["CINGULATE_ALL", "ORBITAL_ALL", "S.T.i.-S.O.T.lat.", "CINGULATE", "S.Or."]

In [43]:
crop_dirs = [f for f in crop_dirs if not os.path.basename(f) in crop_drops]

In [44]:
print(ref_file)
print('\n'.join(crop_dirs))

/neurospin/dico/jchavas/Runs/70_self-supervised_two-regions/Program/2023_jlaval_STSbabies/contrastive/notebooks/joel/utils/../../../configs/dataset/with_reskel_distbottom/2mm/UKB/reference.yaml
/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/crops/2mm/S.F.inf.-BROCA-S.Pe.C.inf.
/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/crops/2mm/S.Po.C.
/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/crops/2mm/F.I.P.
/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/crops/2mm/S.T.s.-S.GSM.
/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/crops/2mm/CINGULATE.
/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/crops/2mm/F.C.L.p.-S.GSM.
/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/crops/2mm/S.C.-S.Po.C.
/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/crops/2mm/F.C.M.post.-S.p.C.
/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/crops/2mm/S.F.int.-F.C.M.ant.
/neurospin/dico/data/deep_foldin

In [45]:
# Read in the reference file
with open(ref_file, 'r') as file:
  ref = file.read()

print(ref)

# @package dataset.REPLACE_DATASET
dataset_name: REPLACE_DATASET
pickle_normal: ${dataset_folder}/UkBioBank/crops/2mm/REPLACE_CROP_NAME/mask/REPLACE_SIDEskeleton.pkl
numpy_all: ${dataset_folder}/UkBioBank/crops/2mm/REPLACE_CROP_NAME/mask/REPLACE_SIDEskeleton.npy
subjects_all: ${dataset_folder}/UkBioBank/crops/2mm/REPLACE_CROP_NAME/mask/REPLACE_SIDEskeleton_subject.csv
crop_dir: ${dataset_folder}/UkBioBank/crops/2mm/REPLACE_CROP_NAME/mask/REPLACE_SIDEcrops
foldlabel_dir: ${dataset_folder}/UkBioBank/crops/2mm/REPLACE_CROP_NAME/mask/REPLACE_SIDElabels
foldlabel_all: ${dataset_folder}/UkBioBank/crops/2mm/REPLACE_CROP_NAME/mask/REPLACE_SIDElabel.npy
subjects_foldlabel_all: ${dataset_folder}/UkBioBank/crops/2mm/REPLACE_CROP_NAME/mask/REPLACE_SIDElabel_subject.csv
distbottom_dir: ${dataset_folder}/UkBioBank/crops/2mm/REPLACE_CROP_NAME/mask/REPLACE_SIDEdistbottom
distbottom_all: ${dataset_folder}/UkBioBank/crops/2mm/REPLACE_CROP_NAME/mask/REPLACE_SIDEdistbottom.npy
subjects_distbottom_all: ${d

In [57]:

def replace_reference_yaml(crop_dir, side, ref):
  """For each crop name, it builds the yaml from the reference yaml"""
  crop_name = os.path.basename(crop_dir)
  dataset_name = crop_name.replace('.', '')
  mask_file = f"{crop_dir}/mask/{side}mask_cropped.nii.gz.minf"
  with open(mask_file, 'r') as file:
    mask = file.read()
  mask = mask.replace("attributes = ", "")
  mask = mask.replace("\'", "\"")
  # print(mask)
  # mask_json = json.loads(mask)
  # print(mask_json)
  side = 'L'
  filedata = ref.replace('REPLACE_CROP_NAME', crop_name)
  filedata = filedata.replace('REPLACE_DATASET', dataset_name)
  filedata = filedata.replace('REPLACE_SIDE', side)
  filedata = filedata.replace('REPLACE_SIZEX', str(mask_json['sizeX']))
  filedata = filedata.replace('REPLACE_SIZEY', str(mask_json['sizeY']))
  filedata = filedata.replace('REPLACE_SIZEZ', str(mask_json['sizeZ']))

  result_file = f"{path}/{dataset_name}"

  return filedata, result_file

In [58]:
# Replace the target string
for crop_dir in crop_dirs:
    for side in ['L', 'R']:
        filedata, result_file = replace_reference_yaml(crop_dir, side, ref)
        

# @package dataset.SFinf-BROCA-SPeCinf
dataset_name: SFinf-BROCA-SPeCinf
pickle_normal: ${dataset_folder}/UkBioBank/crops/2mm/S.F.inf.-BROCA-S.Pe.C.inf./mask/Lskeleton.pkl
numpy_all: ${dataset_folder}/UkBioBank/crops/2mm/S.F.inf.-BROCA-S.Pe.C.inf./mask/Lskeleton.npy
subjects_all: ${dataset_folder}/UkBioBank/crops/2mm/S.F.inf.-BROCA-S.Pe.C.inf./mask/Lskeleton_subject.csv
crop_dir: ${dataset_folder}/UkBioBank/crops/2mm/S.F.inf.-BROCA-S.Pe.C.inf./mask/Lcrops
foldlabel_dir: ${dataset_folder}/UkBioBank/crops/2mm/S.F.inf.-BROCA-S.Pe.C.inf./mask/Llabels
foldlabel_all: ${dataset_folder}/UkBioBank/crops/2mm/S.F.inf.-BROCA-S.Pe.C.inf./mask/Llabel.npy
subjects_foldlabel_all: ${dataset_folder}/UkBioBank/crops/2mm/S.F.inf.-BROCA-S.Pe.C.inf./mask/Llabel_subject.csv
distbottom_dir: ${dataset_folder}/UkBioBank/crops/2mm/S.F.inf.-BROCA-S.Pe.C.inf./mask/Ldistbottom
distbottom_all: ${dataset_folder}/UkBioBank/crops/2mm/S.F.inf.-BROCA-S.Pe.C.inf./mask/Ldistbottom.npy
subjects_distbottom_all: ${dataset_fol

In [47]:
# # Write the file out again
# with open('file.txt', 'w') as file:
#   file.write(filedata)